# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,susuman,62.7833,148.1667,273.96,55,51,7.84,RU,1727134932
1,1,el calafate,-50.3408,-72.2768,274.74,79,66,6.68,AR,1727134685
2,2,blackmans bay,-43.0167,147.3167,283.00,62,82,1.34,AU,1727134935
3,3,port lincoln,-34.7333,135.8667,287.98,73,100,2.52,AU,1727134936
4,4,saint-pierre,-21.3393,55.4781,292.97,73,0,1.54,RE,1727134937


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
humidity = city_data_df["Humidity"].astype(int)

map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)


# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
fix_city_data = city_data_df.loc[(city_data_df['Max Temp']< 27) & (city_data_df['Max Temp']> 21)]

#Wind speed less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]

#Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop results with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,saint-pierre,-21.3393,55.4781,292.97,73,0,1.54,RE,1727134937
33,33,graham,47.0529,-122.2943,297.42,67,0,2.06,US,1727134972
61,61,anadyr,64.7500,177.4833,279.82,65,0,4.00,RU,1727135004
69,69,rawala kot,33.8578,73.7604,291.64,61,0,2.09,PK,1727135014
71,71,port elizabeth,-33.9180,25.5701,289.44,97,0,2.06,ZA,1727135018
...,...,...,...,...,...,...,...,...,...,...
542,542,choix,26.7092,-108.3219,310.17,22,0,3.44,MX,1727135585
543,543,chos malal,-37.3781,-70.2709,282.98,50,0,4.09,AR,1727135586
546,546,fortaleza,-3.7227,-38.5247,300.97,84,0,4.12,BR,1727135392
561,561,shiraz,29.6036,52.5388,289.84,42,0,0.00,IR,1727135609


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["Lat", "Lng", "City", "Country"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df.head()

/var/folders/8y/rss1w25n6knddxjqd8zgc4v80000gn/T/ipykernel_52630/279158293.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,Lat,Lng,City,Country,Hotel Name
4,-21.3393,55.4781,saint-pierre,RE,
33,47.0529,-122.2943,graham,US,
61,64.7500,177.4833,anadyr,RU,
69,33.8578,73.7604,rawala kot,PK,
71,-33.9180,25.5701,port elizabeth,ZA,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint-pierre - nearest hotel: Tropic Hotel
graham - nearest hotel: No hotel found
anadyr - nearest hotel: Гостевой дом
rawala kot - nearest hotel: No hotel found
port elizabeth - nearest hotel: Waterford Hotel
plover - nearest hotel: Hampton
afaahiti - nearest hotel: Omati Lodge
ladnun - nearest hotel: No hotel found
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
kingman - nearest hotel: Home2 Suites by Hilton Kingman
mulayjah - nearest hotel: No hotel found
pacific grove - nearest hotel: Pacific Grove Inn
coquimbo - nearest hotel: Hotel Iberia
vila velha - nearest hotel: Hotel Prainha
constantia - nearest hotel: Hotel Dali
guiyang - nearest hotel: 锦江之星
akureyri - nearest hotel: Hótel Norðurland
merritt island - nearest hotel: No hotel found
syumsi - nearest hotel: No hotel found
colonia - nearest hotel: Wasserturm Hotel Cologne
timimoun - nearest hotel: أناف
remire-montjoly - nearest hotel: Complexe Belova
constitucion - nearest hotel: Hos

,Lat,Lng,City,Country,Hotel Name
4,-21.3393,55.4781,saint-pierre,RE,Tropic Hotel
33,47.0529,-122.2943,graham,US,No hotel found
61,64.7500,177.4833,anadyr,RU,Гостевой дом
69,33.8578,73.7604,rawala kot,PK,No hotel found
71,-33.9180,25.5701,port elizabeth,ZA,Waterford Hotel
...,...,...,...,...,...
542,26.7092,-108.3219,choix,MX,No hotel found
543,-37.3781,-70.2709,chos malal,AR,Hospedaje Lemus
546,-3.7227,-38.5247,fortaleza,BR,Hotel Catedral
561,29.6036,52.5388,shiraz,IR,هتل نصیرالملک


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)